In [2]:
import yaml
import inspect
import napari_workflows as nw
from napari.types import ImageData
from napari_segment_blobs_and_things_with_membranes import gaussian_blur
from skimage.io import imread


w = nw.Workflow()
w.set('denoised',gaussian_blur,'input')
w.set('input',imread("C:/Users/ryans/Desktop/blobs.tif"))

function = w._tasks['denoised'][0]    
args = [None] + list(w._tasks['denoised'][2:])

sig = inspect.signature(function)
print(sig)

(image: napari.types.ImageData, sigma: float = 1, viewer: napari.viewer.Viewer = None) -> napari.types.ImageData


In [9]:
kwargs = {'sigma':4}
bound = sig.bind_partial(**kwargs)
bound

<BoundArguments (sigma=4)>

In [4]:
# code from robert
from inspect import Parameter, Signature, signature
def _generate_signature_for_workflow_step(func, workflow_parameters) -> Signature:
    """Create an inspect.Signature object representing a cle Category.
    The output of this function can be used to set function.__signature__ so that
    magicgui can convert it into the appropriate widget.
    """
    
    
    k = Parameter.KEYWORD_ONLY

    # add inputs (we name them inputN ...)
    params = [
        Parameter(f"input{n}", k, annotation=t) for n, t in enumerate(category.inputs)
    ]
    # Add valid operations choices (will create the combo box)
    from .._categories import operations_in_menu
    choices = list(operations_in_menu(category, search_string))
    #print("choices:", choices)
    op_type = Annotated[str, {"choices": choices, "label": "Operation"}]
    default_op = category.default_op
    if not any(default_op == op for op in choices):
        #print("Default-operation is not in list!")
        default_op = None

    if default_op is None:
        params.append(
            Parameter(OP_NAME_PARAM, k, annotation=op_type)
        )
    else:
        params.append(
            Parameter(OP_NAME_PARAM, k, annotation=op_type, default=default_op)
        )
    # add the args that will be passed to the cle operation.
    for i, (name, type_, default) in enumerate(category_args):
        if i < len(category.default_values):
            default = category.default_values[i]
        params.append(Parameter(name, k, annotation=type_, default=default))

    # add a viewer.  This allows our widget to know if it's in a viewer
    params.append(
        Parameter(VIEWER_PARAM, k, annotation="napari.viewer.Viewer", default=None)
    )
    result = Signature(params)
    #print("Signature", result)
    return result